## Importing Pre-requisite Packages and Dataset

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import re

: 

In [ ]:
data = pd.read_csv("Global_Superstore2.csv")

## Data Understanding

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data.nunique()

In [ ]:
data.select_dtypes(include=np.number).head()

In [ ]:
data.select_dtypes(include=np.object).head()

#### Observations -

- The data has approx 51000 records and 24 variables.
- Within the variables, 7 have been identified as numerical variables.
- Remaining 17 have been identified as categorical
- Postal code is the only columns consisting of missing values.
- Out of the 7, row number and postal code should be identified as categorical as they cannot be used for any aggregations or arithematic operations.
- Also, order date and ship date have been identified as categorical fields whereas they should be datetime fields.

## Data Cleaning and Preparation

In [ ]:
data.columns

In [ ]:
## Removing unwanted spaces and dashes in column names for easier indexing
data.columns = [i.replace(" ","") for i in data.columns]
data.columns = [i.replace("-","") for i in data.columns]

In [ ]:
data.columns

In [ ]:
## Converting row id to categorical / object datatype
data.RowID = data.RowID.astype("object")

In [ ]:
## Converting postal code to categorical / object datatype
data.PostalCode = data.PostalCode.astype("object")

In [ ]:
## Ship date and order date converted to datetime
data.ShipDate = pd.to_datetime(data.ShipDate)
data.OrderDate = pd.to_datetime(data.OrderDate)

In [ ]:
data.select_dtypes(include=np.number).head()

In [ ]:
data.select_dtypes(include=np.object).head()

In [ ]:
## Dropping row id as it is not of any use.
data.drop("RowID",axis=1,inplace=True)

In [ ]:
## Checking for lead spaces
for col in data.select_dtypes(include=np.object).drop("PostalCode",axis=1):
    for i in data[col]:
        if i.strip() != i:
            print(col)
            break

In [ ]:
data.ProductName = data.ProductName.apply(lambda x:x.strip())

In [ ]:
## Getting rid of unneccessary characters causing problems in encoding when needed to export to mysql.
data.ProductName = data.ProductName.apply(lambda x:"".join(re.findall(r"[A-Za-z0-9- ]+",x)))

In [ ]:
## Getting rid of unneccessary characters causing problems in encoding when needed to export to mysql.
data.CustomerName = data.CustomerName.apply(lambda x:"".join(re.findall(r"[A-Za-z0-9- ]+",x)))

In [ ]:
## Getting rid of Postal code as Place information given
data.drop("PostalCode",axis=1,inplace=True)

In [ ]:
## Getting rid of state and city as we will be using region and country for geographical charts or analysis
data.drop(["State","City"],axis=1,inplace=True)

In [ ]:
## Finding time elapsed between date of order and date when order is finally shipped to customer.
data["DaysTillShipment"] = data.ShipDate - data.OrderDate
data.head()

In [ ]:
data.DaysTillShipment = data.DaysTillShipment.apply(lambda x:x.days)
data.head()

In [ ]:
## Creating a new feature to find out whether order was placed on a weekday or weekend.
data["OrderDay"] = data.OrderDate.apply(lambda x:"Weekend Order" if x.weekday() >= 5 else "Weekday Order")
data.head()

In [ ]:
data.Region.unique()

In [ ]:
data.Market.unique()

#### Observation - 
- Within the region variable, we have more than 10 unique levels, which could make it slightly difficult for analysis and visualization. 
- Also there could be several overlapping between central and EMEA regions.
- Within market, we have categories, which could be overlapping logically (ex - EU & EMEA) and many don't make sense (ex - different categories for US and Canada though there are a part of north america).
- Since we wish to focus on region, we will create new categories for regions and replace existing ones.
- We will use the regions assigned to countries in the world indicators dataset.
- We will also use the dataset to replace the country name in the global superstores dataset with the names in the world indicators dataset as the word indicators dataset country name are known to be accepted in tableau geograhical mappings.
- We will drop market as we don't intend to use it and it is similar to regions.

In [ ]:
data.drop("Market",axis=1,inplace=True)

## Creating New Region codes using World Indicators Dataset

In [ ]:
wd = pd.read_csv("WDICountry.csv")

In [ ]:
wd = wd[['Country Code', 'Short Name','Long Name','Region']]

In [ ]:
wd.head()

In [ ]:
wd["Short Name"].sort_values().tail(30)

In [ ]:
data_countries = pd.DataFrame({"Country":data["Country"].unique()})

In [ ]:
data_countries = data_countries.merge(wd,how="left",left_on="Country",right_on="Short Name")

In [ ]:
data_countries[data_countries.isnull().any(axis=1)]

#### Observation - 

- We have successfully replaced the region for all but 14 countries. 
- Since their names are not the same as given in world indicators dataset, they are not finding a match.
- Hence, we will use the country code or short name and manually search and fill the region information

In [ ]:
wd.head()

In [ ]:
try:
    print(wd[wd["Country Code"] == "TWN"])
    data_countries.iloc[16,1:] = wd[wd["Country Code"] == "TWN"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
data_countries.iloc[16,1:] = ["TWN","Taiwan",np.nan,"East Asia & Pacific"]

In [ ]:
print(wd[wd["Short Name"] == "Myanmar"])
data_countries.iloc[73,1:] = wd[wd["Short Name"] == "Myanmar"]

In [ ]:
print(wd[wd["Short Name"] == "Congo"])
data_countries.iloc[[19,76],1:] = wd[wd["Short Name"] == "Congo"]

In [ ]:
try:
    print(wd[wd["Country Code"] == "COD"])
    data_countries.iloc[19,1:] = wd[wd["Country Code"] == "COD"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "COG"])
    data_countries.iloc[76,1:] = wd[wd["Country Code"] == "COG"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "KOR"])
    data_countries.iloc[79,1:] = wd[wd["Country Code"] == "KOR"]

except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "CIV"])
    data_countries.iloc[82,1:] = wd[wd["Country Code"] == "CIV"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "MTQ"])
    data_countries.iloc[90,1:] = wd[wd["Country Code"] == "MTQ"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
data_countries.iloc[90,1:] = ["MTQ","Martinique",np.nan,"Latin America & Caribbean"]

In [ ]:
try:
    print(wd[wd["Country Code"] == "SYR"])
    data_countries.iloc[91,1:] = wd[wd["Country Code"] == "SYR"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
print(wd[wd["Country Code"] == "HKG"])
data_countries.iloc[121,1:] = wd[wd["Country Code"] == "HKG"]

try:
    print(wd[wd["Country Code"] == "HKG"])
    data_countries.iloc[121,1:] = wd[wd["Country Code"] == "HKG"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "GLP"])
    data_countries.iloc[123,1:] = wd[wd["Country Code"] == "GLP"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
data_countries.iloc[123,1:] = ["GLP","Guadeloupe",np.nan,"Latin America & Caribbean"]

In [ ]:
try:
    print(wd[wd["Country Code"] == "KGZ"])
    data_countries.iloc[124,1:] = wd[wd["Country Code"] == "KGZ"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "SWZ"])
    data_countries.iloc[134,1:] = wd[wd["Country Code"] == "SWZ"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
print(wd[wd["Country Code"] == "SVK"])
data_countries.iloc[139,1:] = wd[wd["Country Code"] == "SVK"]

try:
    print(wd[wd["Country Code"] == "SVK"])
    data_countries.iloc[139,1:] = wd[wd["Country Code"] == "SVK"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
try:
    print(wd[wd["Country Code"] == "MKD"])
    data_countries.iloc[143,1:] = wd[wd["Country Code"] == "MKD"]
except ValueError:
    print('Country info doesnt exist in World Indicators Dataset')

In [ ]:
data_countries[data_countries["Region"].isnull()]

In [ ]:
## Replacing the Region with new regions from World Indicators dataset

new_regions = data_countries[["Country","Region","Short Name"]]

new_regions.columns = ["Country","NewRegion","CountryName"]

data = data.merge(new_regions,left_on="Country",right_on="Country")

data.drop(["Region","Country"],axis=1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.NewRegion.value_counts()

In [ ]:
data["Region"] = data.NewRegion

data.drop("NewRegion",axis=1,inplace=True)

In [ ]:
data["Country"] = data.CountryName

data.drop("CountryName",axis=1,inplace=True)

## Transforming dataset for RFM Clustering

#### NOTE - 

- For the purpose of clustering, we will transform the data using the RFM Framework - Recency, Frequency and Monetary value of customers.
- Recency : The days between the last date of order and last date in the dataset
- Frequency : The no. of orders placed.
- Monetary : The total value of all orders placed.

In [ ]:
data.head()

In [ ]:
data.sort_values(by="CustomerName").groupby(["CustomerID","CustomerName"])["Segment"].count().reset_index().sort_values(by="CustomerName").head(50)

#### Important Note - 

- We observe that several customers with same name of 2 different customer ids.
- This is happening due to the inconsistent nature of the customer ID format which removes or include a "10" before the 3 digit number.
- Hence, in order to perform group by, we will use customer name.

In [ ]:
customers = pd.DataFrame({"CustomerName":data.CustomerName.unique()})
customers.head()

In [ ]:
recency = data.groupby("CustomerName").agg({"OrderDate":"max"}).apply(lambda x:(data.OrderDate.max() - x)).reset_index()
recency.head()

In [ ]:
customers = pd.merge(customers,recency,left_on="CustomerName",right_on="CustomerName")
customers.head()

In [ ]:
frequency = data.drop_duplicates("OrderID",keep="first").groupby("CustomerName").agg({"OrderID":"count"}).reset_index()

In [ ]:
customers = pd.merge(customers,frequency,left_on="CustomerName",right_on="CustomerName")
customers.head()

In [ ]:
monetary = data.groupby("CustomerName").agg({"Sales":"sum"}).reset_index()
monetary.head()

In [ ]:
customers = customers.merge(monetary,left_on="CustomerName",right_on="CustomerName")
customers.head()

In [ ]:
customers.columns = ["CustomerName","Recency","Frequency","Monetary"]

In [ ]:
customers.head()

In [ ]:
customers.info()

In [ ]:
customers.Recency = customers.Recency.apply(lambda x:x.days)

In [ ]:
customers.head()

In [ ]:
customers.set_index("CustomerName",inplace=True)
customers.head()

In [ ]:
customers.skew()

In [ ]:
plt.figure(figsize=(12,8))
for index,i in enumerate(customers.columns):
    plt.subplot(2,2,index+1)
    plt.tight_layout(pad=2,h_pad=2)
    plt.title(f"Distribution of {i}")
    sns.distplot(customers[i])

In [ ]:
plt.figure(figsize=(12,8))
for index,i in enumerate(customers.columns):
    plt.subplot(2,2,index+1)
    plt.tight_layout(pad=2,h_pad=2)
    plt.title(f"Distribution of {i}")
    sns.boxplot(customers[i])

In [ ]:
sns.pairplot(customers)

#### Observations - 

- On visualizing the recency, frequency and monetary variables, we observe a lot of highly positive / extreme outliers in the recency variables and some in the frequency variables.
- On visualizing the relationship between the 3 variables using pairplots, we can see that the outliers are resulting in a very wide spread distributions and patterns which could possily affect the quality of the potential clusters.
- Hence, we will first try remove the outliers

In [ ]:
X = customers.copy()
X.head()

In [ ]:
## Removing outliers using power transformer.

from sklearn.preprocessing import StandardScaler,PowerTransformer

PT = PowerTransformer().fit(X)
X = pd.DataFrame(PT.transform(X),columns=X.columns)
X.head()

In [ ]:
plt.figure(figsize=(12,8))
for index,i in enumerate(X.columns):
    plt.subplot(2,2,index+1)
    plt.tight_layout(pad=2,h_pad=2)
    plt.title(f"Distribution of {i}")
    sns.distplot(X[i])

In [ ]:
plt.figure(figsize=(12,8))
for index,i in enumerate(X.columns):
    plt.subplot(2,2,index+1)
    plt.tight_layout(pad=2,h_pad=2)
    plt.title(f"Distribution of {i}")
    sns.boxplot(X[i])

In [ ]:
sns.pairplot(X)

In [ ]:
SC = StandardScaler().fit(X)
X = pd.DataFrame(SC.transform(X),columns=X.columns)
X.head()

## Creating Clusters

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.metrics import silhouette_score


from scipy.cluster.hierarchy import cophenet,dendrogram,linkage
from scipy.spatial.distance import pdist

### Kmeans Clustering

In [ ]:
Kmeans = KMeans(random_state=0).fit(X)
print("Inertia of Kmeans with default 8 clusters:",Kmeans.inertia_)

In [ ]:
inertias = pd.DataFrame({"N_clusters":None,"Inertia":None},index=range(1))

for i in range(2,11):
    kmeans = KMeans(n_clusters=i,random_state=0).fit(X)
    inertias = inertias.append({"N_clusters":i,"Inertia":kmeans.inertia_},ignore_index=True)
    

In [ ]:
inertias.dropna(inplace=True)

plt.figure(figsize = (8,4))
plt.plot(inertias.N_clusters,inertias.Inertia,marker="o")
plt.xlabel("No. of Clusters")
plt.ylabel("Inertia")
plt.title("Elbow Plot",size=15)
plt.show()

In [ ]:
inertias["Silhoutte_Score"] = None
scores = []

for i in range(2,11):
    kmeans = KMeans(n_clusters=i,random_state=0).fit(X)
    score = silhouette_score(X,kmeans.labels_)
    scores.append(score)
    
inertias["Silhoutte_Score"] = scores

In [ ]:
inertias.sort_values("Silhoutte_Score",ascending=False)

In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

for i in range(2,11):
    kmeans = KMeans(n_clusters=i,random_state=0)
    visualizer = SilhouetteVisualizer(kmeans, colors='yellowbrick')
    visualizer.fit(X)
    visualizer.show()

### Agglomerative Clustering
  

In [ ]:
Agg = AgglomerativeClustering().fit(X)
score = silhouette_score(X,Agg.labels_)
print("Silhoutte score for Agglomerative clustering with default cluster i.e 2:",score)

In [ ]:
agg_scores = pd.DataFrame({"N_clusters":None,"Silhoutte":None},index=range(1))

for i in range(2,11):
    agg = AgglomerativeClustering(n_clusters=i).fit(X)
    score = silhouette_score(X,agg.labels_)
    agg_scores = agg_scores.append({"N_clusters":i,"Silhoutte":score},ignore_index=True)
    

In [ ]:
agg_scores.dropna(inplace=True)
agg_scores

In [ ]:
for met in ["single","average","ward","complete"]:
    for dist in ["euclidean","minkowski"]:
        if met != "ward":
            distance = linkage(X,method=met,metric=dist)
            cop = cophenet(distance,pdist(X,metric=dist))
            print(f"Cophenet coefficient for {met}-{dist}:",cop[0])
        else:
            distance = linkage(X,method=met,metric="euclidean")
            cop = cophenet(distance,pdist(X,metric="euclidean"))
            print(f"Cophenet coefficient for {met}-euclidean:",cop[0])

In [ ]:
for metric in ["single","average","ward","complete"]:
    distance = linkage(X,method=metric)
    plt.figure(figsize=(10,10))
    dendrogram(distance)
    plt.title(f"Dendogram Using {metric} method")

In [ ]:
for met in ["single","average","ward","complete"]:
    for i in [2,3]:
        agg = AgglomerativeClustering(n_clusters=i,linkage=met).fit(X)
        score = silhouette_score(X,agg.labels_)
        print(f"silhoutte coefficient with {i} cluster for {met} using euclidean distance:",score)

#### Observations
- On using Kmeans clusterings, we observed that when n cluster = 2, we get the highest silhouette score i.e 0.30.
- On using Agglomerative clustering, we observed that when n cluster = 2, we get the highest silhouette score i.e 0.25 when using ward method of distance calculation.
- We did get better scores in agglomerative using single and average method of distance calculation. However, the clusters were not clearly distinguishable since very few customers were put into 1 cluster.
- Also, on using the ward method for calculating dendogramic distances, we observed a good differentiation between data points when n-clusters = 2.

**Hence, we choose n-clusters = 2** 

### Visualizing the new clusters

In [ ]:
kmeans = KMeans(n_clusters=2).fit(X)
agg = AgglomerativeClustering(n_clusters=2,linkage="ward").fit(X)
customers["Kmeans_Cluster"] = kmeans.labels_
customers["Agg_Cluster"] = kmeans.labels_


In [ ]:
customers["Kmeans_Cluster"] = customers.Kmeans_Cluster.apply(lambda x:f"Cluster {x}")
customers["Agg_Cluster"] = customers.Agg_Cluster.apply(lambda x:f"Cluster {x}")

In [ ]:
sns.pairplot(customers,hue="Kmeans_Cluster")

In [ ]:
sns.pairplot(customers,hue="Agg_Cluster")
plt.show()

In [ ]:
customers.drop("Agg_Cluster",axis=1,inplace=True)

In [ ]:
customers

In [ ]:
plt.figure(figsize=(12,8))
for index,i in enumerate(customers.columns[:-1]):
    plt.subplot(2,2,index+1)
    plt.tight_layout(pad=2,h_pad=2)
    plt.title(f"Cluster 0 vs 1: Distribution of {i}")
    sns.boxplot(y=customers[i],x=customers.Kmeans_Cluster)

In [ ]:
customers[customers.Kmeans_Cluster=="Cluster 0"].describe()

In [ ]:
customers[customers.Kmeans_Cluster=="Cluster 1"].describe()

#### Observations - 

- On visualizing the kmeans and agglomerative clusters formed seperately and comparing, we see no difference in patterns.
- Hence, we can use either of the two.
- Also, on comparing the RFM features for the 2 different clusters, we find many differences.
- Based on medians, the recency value for customers in cluster 0 is 22.5 whereas in cluster 1 its 10.
- Based on medians, the frequency value for customers in cluster 0 is 28 whereas in cluster 1 its 35.
- Based on medians, the monetary value for customers in cluster 0 is around 12500 whereas in cluster 1 its around 22000.

**These observations show that the differences observed by the means of the clusters are good enough to differentiate the customers into 2 groups.**

In [ ]:
customers.reset_index(inplace=True)

In [ ]:
customer_cluster = customers[["CustomerName","Kmeans_Cluster"]]

In [ ]:
data = data.merge(customer_cluster,left_on="CustomerName",right_on="CustomerName")

In [ ]:
data.head()

## Exporting File to Sales Table in MySQL

In [ ]:
from sqlalchemy import create_engine

In [ ]:
my_conn = create_engine("mysql+mysqldb://root:06MYSQLkey@#@localhost/python")

In [ ]:
data.head()

In [ ]:
data.columns

In [ ]:
data.to_sql(con=my_conn,name="sales",if_exists="replace",index=False)